<a href="https://colab.research.google.com/github/matthewroche/house_prices/blob/main/House%20Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [154]:
import kaggle
!kaggle competitions download house-prices-advanced-regression-techniques

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
data_description.txt: Skipping, found more recently modified local copy (use --force to force download)


In [367]:
import pandas as pd

train = pd.read_csv("train.csv", na_filter=False)

print(train.shape)
print(train.columns)

(1460, 81)
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 

In [302]:
# Check column types
print(pd.DataFrame(train.dtypes).to_string())

                    0
Id              int64
MSSubClass      int64
MSZoning       object
LotFrontage    object
LotArea         int64
Street         object
Alley          object
LotShape       object
LandContour    object
Utilities      object
LotConfig      object
LandSlope      object
Neighborhood   object
Condition1     object
Condition2     object
BldgType       object
HouseStyle     object
OverallQual     int64
OverallCond     int64
YearBuilt       int64
YearRemodAdd    int64
RoofStyle      object
RoofMatl       object
Exterior1st    object
Exterior2nd    object
MasVnrType     object
MasVnrArea     object
ExterQual      object
ExterCond      object
Foundation     object
BsmtQual       object
BsmtCond       object
BsmtExposure   object
BsmtFinType1   object
BsmtFinSF1      int64
BsmtFinType2   object
BsmtFinSF2      int64
BsmtUnfSF       int64
TotalBsmtSF     int64
Heating        object
HeatingQC      object
CentralAir     object
Electrical     object
1stFlrSF        int64
2ndFlrSF  

In [368]:
# Split inputs and outputs
output = train["SalePrice"]
train = train.drop(["SalePrice"], axis=1)

In [369]:
# Notice lot frontage has been processed as a string as it contains "NA"s
# Replace NA's with mean

def handleColumnWithNA(df, colName, **means):
  df[colName] = pd.to_numeric(df[colName], errors="coerce", downcast="float")
  if colName in means:
    df[colName].fillna(means[colName], inplace = True)
  else:
    df[colName].fillna(df[colName].mean(), inplace = True)
  return df

train = handleColumnWithNA(train, "LotFrontage")

means = train.mean(axis=0)

In [370]:
# Drop ID column
train.drop(labels="Id", axis=1, inplace=True)

In [371]:
# Check for other NaN

nanCols = []

for col in train.columns:
  if train[col].isnull().values.any():
    nanCols.append(col)
    print(f"Column: {col}")
    print(train[col].isnull().sum())
    print(train[col][train[col].isnull()])

In [373]:
# Wide range of means and standard deviations - lets adjust

trainBeforeNormalisation = train

import numpy as np
from sklearn import preprocessing
from pandas.api.types import is_numeric_dtype
min_max_scaler = preprocessing.MinMaxScaler()

def performNumericScaling(df, initial):

  # First get only the columns containing numeric data
  numericCols = []
  for col in test.columns:
    if is_numeric_dtype(test.dtypes[col]):
      numericCols.append(col)

  # Build a new data frame with just these columns
  numericDf = df[numericCols]

  # Do the scaling
  x = numericDf.values #returns a numpy array
  if initial:
    min_max_scaler.fit(x)
  x_scaled = min_max_scaler.transform(x)
  numericDf = pd.DataFrame(x_scaled, columns=numericDf.columns)

  # Put the scaled columns back into our original data frame
  for col in numericCols:
    df.drop([col], axis=1)
    df[col] = numericDf[col]

  return df

train = performNumericScaling(train, True)

In [374]:
# Encode strings

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=True)

def performOneHotEncoding(df, initial):

  strCols = []
  for col in df.columns:
    if df.dtypes[col] == object:
      strCols.append(col)

  # Build a new data frame with just these columns
  stringDf = df[strCols]
  
  if initial:
    enc.fit(stringDf.values)
  fit = enc.transform(stringDf.values).toarray()

  # Not adding column headers as they don't matter
  stringDfEncoded = pd.DataFrame(fit)

  df.drop(labels=strCols, axis=1, inplace=True)
  df = df.join(stringDfEncoded)
  return df
  
train = performOneHotEncoding(train, True)

In [383]:
# Train a neural net
print(train.shape)
from sklearn.neural_network import MLPRegressor
clf = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(train, output)

(1460, 728)


MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(5, 2), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [385]:
# Load test data
test = pd.read_csv("test.csv", na_filter=False, dtype=object)

# Save IDs
IDs = test["Id"]
test = test.drop(labels=["Id"], axis=1)

In [386]:
# Compare train / test data types
trainDTypes = trainBeforeNormalisation.dtypes
trainDTypes.name = "Train"
testDTypes = test.dtypes
testDTypes.name = "Test"
print(pd.concat((trainDTypes, testDTypes), axis=1).to_string())

                 Train    Test
MSSubClass     float64  object
LotFrontage    float64  object
LotArea        float64  object
OverallQual    float64  object
OverallCond    float64  object
YearBuilt      float64  object
YearRemodAdd   float64  object
BsmtFinSF1     float64  object
BsmtFinSF2     float64  object
BsmtUnfSF      float64  object
TotalBsmtSF    float64  object
1stFlrSF       float64  object
2ndFlrSF       float64  object
LowQualFinSF   float64  object
GrLivArea      float64  object
BsmtFullBath   float64  object
BsmtHalfBath   float64  object
FullBath       float64  object
HalfBath       float64  object
BedroomAbvGr   float64  object
KitchenAbvGr   float64  object
TotRmsAbvGrd   float64  object
Fireplaces     float64  object
GarageCars     float64  object
GarageArea     float64  object
WoodDeckSF     float64  object
OpenPorchSF    float64  object
EnclosedPorch  float64  object
3SsnPorch      float64  object
ScreenPorch    float64  object
PoolArea       float64  object
MiscVal 

In [387]:
# Process Test Data

# Several of the columns that were processed as numbers before contin "NA" in the test data
# We've therefore imported all columns as strings, and will need to match the column types
for col in trainBeforeNormalisation.columns:
  if test.dtypes[col] != trainBeforeNormalisation.dtypes[col]:
    # As we've imported all the columns as strings, if they don't match this must be because the column was an integer in the training data
    test = handleColumnWithNA(test, col, **means)

# Scale
test = performNumericScaling(test, False)
test = performOneHotEncoding(test, False)

print(test.shape)

(1459, 728)
